In [65]:
import numpy as np

$\renewcommand{\diag}[1]{\text{diag}\left(#1\right)}$

# A simple numerical example on simulated data

## Simulating needed data

For now, I assume production functions are Cobb-Douglass, households have Cobb-Douglas preferences over final consumption goods, and the matching function is Cobb-Douglas. In principle, we don't need to be so restrictive about the functional forms of production and matching. Instead all of the relevant information about production technologies is captured by the levels and changes of the elasticities outline below. Any two production functions with the same elasticities and changes in elasticities would generate the same responses to shocks.

For each sector $j$, we need the elasticity of production of good $j$ with respect to the intermediate input from each other sector $i$, $\varepsilon^{f_j}_{x_{ji}}$, and the elasticity of production with respect to the labor input $N_j$, $\varepsilon^{f_j}_{N_{j}}$. I sample elasticites of production on the unit simplex using an exponential transformation of the normal distribution. To start, for each industry, draw
\begin{align*}
    \left(z_{j1}, \cdots,z_{jJ},z_{N_j}\right) \sim N\left(\mu,\Sigma\right)
\end{align*}
Then define
\begin{align*}
    \varepsilon^{f_j}_{x_{j1}} = \frac{e^{z_{j1}}}{e^{z_{N_j}}+\sum_{k=1}^{J}e^{z_{jk}}}
\end{align*}
And equivalently for all other sectors. We also need the elasticities of demand with respect to sector $j$'s output, $\varepsilon^{\mathcal{D}}_{c_j}$. Notice that we can treat the final demand as another production sector that uses no labor input. We can sample the demand elasticities using the same procedure outlined above.

Finally, I sample the matching elasticity with respect to vacancies from a uniform distribution on $[0,1]$. 

We let $\Omega$ be the matrix of input elasticities for each sector and $I-\Psi$ be the Leontif inverse.

In [66]:
# Size of network
J = 4 

# Sampling input-output matrix entries 
μ = np.zeros(J+1)
Σ = np.eye(J+1)
z_draws = np.random.multivariate_normal(μ, Σ, J+1)
elasticity_draws = np.exp(z_draws)/np.sum(np.exp(z_draws),1).reshape((J+1,1))

# Elasticities
epsD = elasticity_draws[0,:-1]/np.sum(elasticity_draws[0,:-1])
epsD = epsD.reshape(J,1)
Omega = elasticity_draws[1:,:-1]
Psi = np.linalg.inv(np.eye(J)-Omega)
epsN = np.diag(elasticity_draws[1:,-1])

# Drawing elasticity of matching function wrt to U
ν = np.random.uniform(size=J)
curlyQ = np.diag(-ν)
curlyF =  np.eye(J) + curlyQ

Finally, we need recruiter producer ratios $\tau_j(\theta_j)$ in each sector. Landais, Michaillat, and Saez (2018) find that the share of recruiters in the US workforce averages around 2.3 percent. I sample recruiter producer ratios uniformly on the interval $[0,0.046]$ to roughly match this fact.

In [67]:
tau = np.diag(np.random.uniform(low=0,high=0.046,size=J))
tau

array([[0.02727481, 0.        , 0.        , 0.        ],
       [0.        , 0.02965199, 0.        , 0.        ],
       [0.        , 0.        , 0.01218354, 0.        ],
       [0.        , 0.        , 0.        , 0.00958356]])

## Defining shocks
We are interested in the response of sector level and aggregate output and employment to technology shocks $d\log\bm{A}$ and labor force shocks $d\log \bm{H}$. The code below defines the shocks we feed into the model.

In [68]:
# Technology shocks
dlog_A = 0.01*np.ones(J)
dlog_H = 0.01*np.zeros(J)

## Defining how wages adjust
Since there are mutual gains from trade once an unemployed worker and a firm meet, wages are not pinned down uniquely in models featuring search and matching frictions in the labor market. We must therefore impose a wage schedule, an assumption about how wages change in response to fundamentals, in order to close the model. 

Let $W_i$ be the nominal sector wage and $w_i = W_i / P_i$ the wage for sector i adjusted for goods price in that sector. 

In particular, we need to specify the elasticity of wages to technology shocks and labor force shocks in each sector, $\left\{\left\{\varepsilon^{w_j}_{A_{ji}},\varepsilon^{w_j}_{H_{ji}}\right\}_{i=1}^{J}\right\}_{j=1}^{J}$. For instance, a simple, but unrealistic, assumption is that wages respond positively to own sector productivity and negatively to own sector labor force, but do not respond to changes in other sectors. In general, we express changes in wages as a function of changes in productivity and the labor force. 
\begin{align}
    d\log \bm{w} &= \bm{\Lambda_{A}} d\log \bm{A} + \bm{\Lambda_{H}} d\log \bm{H} \tag{1}
\end{align}
Where $\bm{\Lambda_{A}}$ contains wage elasticities to productivity changes and $\bm{\Lambda_{H}}$ contains wage elasticities to labor force changes.

In [69]:
# Wage elasticities
epsW_A = np.diag(np.random.uniform(low=3,high=7,size=J))
epsW_H = np.diag(np.random.uniform(low=-2,high=0,size=J))
epsW_A

array([[5.68902907, 0.        , 0.        , 0.        ],
       [0.        , 4.71080677, 0.        , 0.        ],
       [0.        , 0.        , 5.81175704, 0.        ],
       [0.        , 0.        , 0.        , 6.30159706]])

In [70]:
# Calculating the log change in wages
def WageFunc(dlog_A, dlog_H, epsW_A, epsW_H):
    dlog_wR = epsW_A @ dlog_A + epsW_H @ dlog_H
    return dlog_wR

In [71]:
#How wages change
dlog_wR = WageFunc(dlog_A, dlog_H, epsW_A, epsW_H)
dlog_wR

array([0.05689029, 0.04710807, 0.05811757, 0.06301597])


## Tightness propagation
We with wage changes in hand, we solve for first order changes in tightness in terms of $d\log\bm{A}$, $d\log\bm{H}$, and $d\log\bm{w}$. The general formula for changes in tightness, treating sector 1 prices as the numeraire, is
\begin{align}
    d \log \bm{\theta} &=\left(\diag{\bm{\mathcal{F}}}-\bm{\Xi_{\theta}}\right)^{-1}\left(\left(\bm{I} - \bm{\Psi} \diag{\bm{\varepsilon^f_N}}\right) \left(d\log \bm{\varepsilon^f_N} + d\log \bm{\lambda} - d\log \bm{H}\right) + \bm{\Psi} d\log \bm{A}\right) \\
    &- \left(\diag{\bm{\mathcal{F}}}-\bm{\Xi_{\theta}}\right)^{-1} \left(d\log \bm{w}-d\log\bm{p}\right) \tag{2}
\end{align}
Where 
\begin{align*}
    \bm{\Xi_\theta} = \bm{\Psi}\diag{\bm{\varepsilon^f_N}}\left(\diag{\bm{\mathcal{F}}} + \diag{\bm{\tau}}\diag{\bm{\varepsilon^{\mathcal{Q}}_{\theta}}}\right)
\end{align*}

In [72]:
def ThetaFunc(dlog_A, dlog_H, dlog_wR, dlog_epsN, dlog_lam, Psi, curlyF, curlyQ, epsN, tau, num=0):
    
    # Creating matrices
    Xi_a = Psi

    Xi_theta = Psi @ epsN @ curlyF+ Psi @ epsN @ tau @ curlyQ
    #Xi_theta[num,:] = Xi_theta[num,:] + Psi[num,:] @ epsN @ tau @ curlyQ

    Xi_w = np.zeros_like(Psi)

    I = np.eye(Psi.shape[0])

    # Contribution of different components
    Cw = np.linalg.inv(curlyF - Xi_theta) @ (I - Xi_w)
    Ca = np.linalg.inv(curlyF - Xi_theta) @ Xi_a 
    Ch = np.linalg.inv(curlyF - Xi_theta) @ (I - Psi @ epsN)

    # Change in tightness
    dlog_theta = Ch @ (dlog_epsN + dlog_lam - dlog_H) + Ca @ dlog_A - Cw @ dlog_wR

    return dlog_theta

Assuming Cobb-Douglas production implies $d\log\bm{\varepsilon^f_N} = d\log \bm{\lambda} = 0$.

In [73]:
dlog_epsN = np.zeros_like(dlog_A)
dlog_lam = np.zeros_like(dlog_A)
dlog_theta = ThetaFunc(dlog_A, dlog_H, dlog_wR, dlog_epsN, dlog_lam, Psi, curlyF, curlyQ, epsN, tau, num=1)
dlog_theta

array([-3.37118205, -3.25595174, -2.66270593, -4.35238829])

## Price and output propagation
With changes in tightness in hand, we can now work out how prices and sectoral production changes in response to technology and labor force shocks. Price changes are given by 
\begin{align}
    \left(I - \Psi\text{diag}\left(\bm{\varepsilon^{f}_{N}}\right)\right)d \log \bm{p} &=\bm{\Psi} \left(
        \text{diag}\left(\bm{\varepsilon^{f}_{N}}\right) \left(d\log\bm{w}  - d\log\bm{p}\right) -\text{diag}\left(\bm{\varepsilon^{f}_{N}}\right)\text{diag}\left(\bm{\tau}\right)\text{diag}\left(\bm{\varepsilon^{\mathcal{Q}}_{\theta}}\right)d\log\bm{\theta} - d\log \bm{A} \right) \tag{3}
\end{align}
While $\Xi = \left(I - \Psi\text{diag}\left(\bm{\varepsilon^{f}_{N}}\right)\right)$ is not invertible, we impose an additional restrction $d\log p_i = 0$, with the good produced by sector $i$ the numeraire good of our choice.

In [74]:
def PriceFunc(dlog_A, dlog_wR, dlog_theta, Psi, curlyQ, epsN, tau, num=0):
    # Contributions of different components
    Cw = Psi @ epsN 
    Cw[num, :] = 0
    Ct = Psi @ epsN @ tau @ curlyQ
    Ct[num, :] = 0
    Ca = np.copy(Psi)
    Ca[num, :] = 0
    
    Xi = np.eye(Psi.shape[0]) - Psi @ epsN 
    Xi[num, :] = 0
    Xi[num, num] = 1

    # Price changes
    dlog_p = np.linalg.inv(Xi) @ (Cw @ dlog_wR - Ct @ dlog_theta - Ca @dlog_A)
    return dlog_p


In [75]:
num = 0
dlog_p = PriceFunc(dlog_A, dlog_wR, dlog_theta, Psi, curlyQ, epsN, tau, num)
dlog_p

array([ 0.        , -0.01103464, -0.00889522,  0.00046135])

Alternatively, we can solve explicitly for $d\log\bm{p}$ 
\begin{align*}
    d\log \bm{p} &= \bm{\Xi_p}^{-1}\text{diag}\left(\bm{\varepsilon^{f}_{N}}\right) \left(d\log\bm{w} - d\log \bm{p}\right)\nonumber \\
    &-\bm{\Xi_p}^{-1}\text{diag}\left(\bm{\varepsilon^{f}_{N}}\right)\text{diag}\left(\bm{\tau}\right)\text{diag}\left(\bm{\varepsilon^{\mathcal{Q}}_{\theta}}\right)d\log\bm{\theta}\\
    &- \bm{\Xi_p}^{-1} d\log \bm{A}
\end{align*}
Where
\begin{align*}
    \bm{\Xi_{p}} &= \begin{bmatrix}
        0 & \bm{0}_{1\times (J-1)} \\
        \bm{0}_{(J-1)\times 1} & \bm{I}_{(J-1) \times (J-1)} - \diag{\bm{\varepsilon^f_N}\setminus\varepsilon^{f_1}_{N_1}}
    \end{bmatrix} - \bm{\Omega}
\end{align*}
The two solution methods should coincide to machine precision. 

In [76]:
def PriceFunc2(dlog_A, dlog_wR, dlog_theta, Omega, curlyQ, epsN, tau, num = 0):
    Xi_p = np.eye(Omega.shape[0]) - epsN 
    Xi_p[num, : ] = 0 
    Xi_p = Xi_p - Omega
    dlog_p = np.linalg.inv(Xi_p) @ (epsN @ (dlog_wR - tau @ curlyQ @ dlog_theta) - dlog_A)
    return dlog_p

In [77]:
dlog_p2 = PriceFunc2(dlog_A, dlog_wR, dlog_theta, Omega, curlyQ, epsN, tau, num)
dlog_p2


array([-5.68121938e-16, -1.10346445e-02, -8.89521568e-03,  4.61351809e-04])

We can check that our solution is consistent with $p_i$ being the numeraire by checking that 
\begin{align*}
    0 & = \varepsilon^{f_i}_{N_i} \left( d\log w_i - d\log p_i - \tau_i(\theta_i) \varepsilon^{\mathcal{Q_i}}_{\theta_i} d\log \theta_i\right)+\Omega_i d\log\bm{p} - d\log A_i 
\end{align*}

In [78]:
epsN[num,num] * (dlog_wR[num] - tau[num,num] * curlyQ[num,num] * dlog_theta[num]) + Omega[num,] @ dlog_p - dlog_A[num]

5.499073418846478e-16

In [79]:
epsN[num,num] * (dlog_wR[num] - tau[num,num] * curlyQ[num,num] * dlog_theta[num]) + Omega[num,] @ dlog_p2 - dlog_A[num]

-1.734723475976807e-18

Finally, we can check that our two solution methods coincide, providing another sanity check on our code.

In [80]:
dlog_p - dlog_p2

array([5.68121938e-16, 1.23512311e-15, 1.23512311e-15, 1.22861790e-15])

And output changes are given by
\begin{align}
    d\log \bm{y} &= \bm{\Psi}\left(d\log\bm{A} + \text{diag}\left(\bm{\varepsilon^{f}_{N}}\right)\left(\text{diag}\left(\bm{\mathcal{F}}\right)+\text{diag}\left(\bm{\tau}\right)\text{diag}\left(\bm{\varepsilon^{\mathcal{Q}}_{\theta}}\right)\right)d\log \bm{\theta} + \text{diag}\left(\bm{\varepsilon^{f}_{N}}\right) d\log\bm{H}\right) \nonumber\\
    &-\bm{\Psi} \text{diag}\left(\bm{\varepsilon^{f}_{N}}\right) d\log \bm{\varepsilon^{f}_{N}} + \left(\bm{I} - \bm{\Psi}\text{diag}\left(\bm{\varepsilon^{f}_{N}}\right)\right) d\log \bm{\lambda} \tag{4}
\end{align}


In [81]:
def OutputFunc(dlog_A, dlog_H, dlog_theta, dlog_epsN, dlog_lam, Psi, curlyQ, curlyF, epsN, tau):
    # Contributions of different coponents
    Ca = Psi
    Ct = Psi @ epsN @ (curlyF + tau @ curlyQ) 
    Ch = Psi @ epsN 
    I = np.eye(Psi.shape[0])

    # Changes in output
    dlog_y = Ca @ dlog_A + Ct @ dlog_theta + Ch @ (dlog_H-dlog_epsN) + (I - Ch) @ dlog_lam 

    return dlog_y

In [82]:
dlog_y = OutputFunc(dlog_A, dlog_H, dlog_theta, dlog_epsN, dlog_lam, Psi, curlyQ, curlyF, epsN, tau)
dlog_y

array([-2.24859038, -2.23755573, -2.23969516, -2.24905173])

With Cobb-Douglas production, real output should change by the same amount in each sector. This provides an overall check of our code given the Cobb-Douglas assumption.

In [83]:
# Change in nominal GDP
dlog_p + dlog_y

array([-2.24859038, -2.24859038, -2.24859038, -2.24859038])

We can now check consistency. We should get the same change in labor by either calculating the change in labor supply
\begin{align}
    d\log \bm{L^s} = \text{diag}\left(\mathcal{F}\right) d\log \bm{\theta} + d\log \bm{H} \tag{5}
\end{align}
Or the change in labor demand
\begin{align}
    d\log \bm{L^d} = d\log \bm{\varepsilon^{f}_N} + d\log \bm{p} + d\log\bm{y} - d\log\bm{w} \tag{6}
\end{align}

In [84]:
def LaborSupply(dlog_H,dlog_theta,curlyF):
    dlog_Ls = curlyF @ dlog_theta + dlog_H
    return dlog_Ls
    
def LaborDemand(dlog_wR, dlog_y, dlog_epsN):
    dlog_Ld = dlog_epsN  + dlog_y - dlog_wR
    return dlog_Ld

Consistency requires
\begin{align*}
    d\log \bm{L^d} - d\log\bm{L^s} = 0
\end{align*}

In [85]:
LaborDemand(dlog_wR, dlog_y, dlog_epsN) - LaborSupply(dlog_H,dlog_theta,curlyF)

array([ 0.0000000e+00, -4.4408921e-16, -4.4408921e-16, -4.4408921e-16])

## Unemployment
We can use changes in tightness and the labor force to calculate changes in employment. 
\begin{align}
    d\log \bm{L} = \text{diag}\left(\mathcal{F}\right) d\log \bm{\theta} + d\log \bm{H} \tag{5}
\end{align}

In [86]:
dlog_L = LaborSupply(dlog_H,dlog_theta,curlyF)
dlog_L

array([-2.30548067, -2.2846638 , -2.29781273, -2.3120677 ])

Unemployment in sector $j$ is $U_j = H_j - L_j$. The log change in Unemployment is 
\begin{align}
    d\log \bm{U} = \diag{\bm{U}}^{-1}\left(\diag{\bm{H}} d\log \bm{H} - \diag{L}d\log\bm{L}\right)\tag{6}
\end{align}
The unemployment rate in sector $j$ is $u_j = \frac{H_j-L_j}{H_j}$. The log change in the unemployment rate is 
\begin{align}
    d\log \bm{u} = \left(\bm{I} - \diag{\bm{u}}\right)\diag{\bm{u}}^{-1} \left(d\log\bm{H} - d\log\bm{L}\right) \tag{7}
\end{align}


In [87]:
def UnemploymentFunc(dlog_H, dlog_L, U, H, L):
    dlog_U = np.linalg.inv(U) @ (H @ dlog_H - L @ dlog_L)
    return dlog_U

def UnRateFunc(dlog_H, dlog_L, u):
    dlog_u = (np.eye(dlog_H.shape[0]) - u) @ np.linalg.inv(u) @ (dlog_H - dlog_L)
    return dlog_u

## Aggregation
Given first order changes in output, we can use the elasticities of demand and changes in domar weights calculate changes in aggregate output.
\begin{align}
    d\log Y = \bm{\varepsilon^{\mathcal{D}}_{c}}' \left(d\log\bm{\varepsilon^{\mathcal{D}}_{c}} + d\log \bm{y} - d\log \bm{\lambda}\right) \tag{8}
\end{align}

In [90]:
def AggOutputFunc(dlog_y, dlog_epsD, dlog_lam, epsD):
    dlog_Y = epsD.T @ (dlog_epsD + dlog_y - dlog_lam)
    return dlog_Y

Assuming Cobb-Douglas, $d\log \bm{\varepsilon^{\mathcal{D}}_{c}} = d\log \bm{\lambda} = \bm{0}$. 

In [91]:
dlog_epsD = np.zeros_like(dlog_A)
dlog_Y = AggOutputFunc(dlog_y, dlog_epsD, dlog_lam, epsD)
dlog_Y

array([-2.24179239])